In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
import tensorflow as tf

# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Rename Columns
charity2_df = charity_df.rename(columns={"APPLICATION_TYPE": "APPTYPE", "USE_CASE": "USECASE", "SPECIAL_CONSIDERATIONS": "SPECCONS", "INCOME_AMT": "INCAMT", "IS_SUCCESSFUL": "SUCCESSFUL"})
charity2_df.head()

,EIN,NAME,APPTYPE,AFFILIATION,CLASSIFICATION,USECASE,ORGANIZATION,STATUS,INCAMT,SPECCONS,ASK_AMT,SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Successful to a String
charity2_df['SUCCESSFUL'] = charity2_df.SUCCESSFUL.astype(str)
charity2_df.head()

,EIN,NAME,APPTYPE,AFFILIATION,CLASSIFICATION,USECASE,ORGANIZATION,STATUS,INCAMT,SPECCONS,ASK_AMT,SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Check to see if bucketing is required (anything over 10)
# Generate our categorical variable list
charity2_cat = charity2_df.dtypes[charity2_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity2_df[charity2_cat].nunique()

NAME              19568
APPTYPE              17
AFFILIATION           6
CLASSIFICATION       71
USECASE               5
ORGANIZATION          4
INCAMT                9
SPECCONS              2
SUCCESSFUL            2
dtype: int64

In [5]:
# Check the unique value counts of Application Type to see if binning is required
APPTYPE_counts = charity2_df.APPTYPE.value_counts()
APPTYPE_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPTYPE, dtype: int64

In [6]:
# Bucketing for APPLICATION_TYPE

# Determine which values to replace
replace_types = list(APPTYPE_counts[APPTYPE_counts < 100].index)

# Replace in DataFrame
for APPTYPE in replace_types:
    charity2_df.APPTYPE = charity2_df.APPTYPE.replace(APPTYPE,"Other")


# Check to make sure binning was successful
charity2_df.APPTYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPTYPE, dtype: int64

In [7]:
# Check the unique value counts of Classification to see if binning is required
CLASS_counts = charity2_df.CLASSIFICATION.value_counts()
CLASS_counts.head(30)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
Name: CLASSIFICATION, dtype: int64

In [8]:
# Bucketing for CLASSIFICATION

# Determine which values to replace
replace_types = list(CLASS_counts[CLASS_counts < 1800].index)

# Replace in DataFrame
for CLASSIFICATION in replace_types:
    charity2_df.CLASSIFICATION = charity2_df.CLASSIFICATION.replace(CLASSIFICATION,"Other")


# Check to make sure binning was successful
charity2_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Check to see if bucketing is required (anything over 10)
# Generate our categorical variable list
charity2_cat = charity2_df.dtypes[charity2_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity2_df[charity2_cat].nunique()

NAME              19568
APPTYPE              10
AFFILIATION           6
CLASSIFICATION        6
USECASE               5
ORGANIZATION          4
INCAMT                9
SPECCONS              2
SUCCESSFUL            2
dtype: int64

In [10]:
# Drop name and EIN column since it will be different for each row and not useful
charity3_df = charity2_df.drop(columns=['NAME', 'EIN'])
charity3_df.head()

,APPTYPE,AFFILIATION,CLASSIFICATION,USECASE,ORGANIZATION,STATUS,INCAMT,SPECCONS,ASK_AMT,SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [11]:
# Check to see if bucketing is required (anything over 10)
# Generate our categorical variable list
charity3_cat = charity3_df.dtypes[charity3_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity3_df[charity3_cat].nunique()

APPTYPE           10
AFFILIATION        6
CLASSIFICATION     6
USECASE            5
ORGANIZATION       4
INCAMT             9
SPECCONS           2
SUCCESSFUL         2
dtype: int64

In [12]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity3_df[charity3_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity3_cat)
encode_df.head()


,APPTYPE_Other,APPTYPE_T10,APPTYPE_T19,APPTYPE_T3,APPTYPE_T4,APPTYPE_T5,APPTYPE_T6,APPTYPE_T7,APPTYPE_T8,APPTYPE_T9,...,INCAMT_100000-499999,INCAMT_10M-50M,INCAMT_1M-5M,INCAMT_25000-99999,INCAMT_50M+,INCAMT_5M-10M,SPECCONS_N,SPECCONS_Y,SUCCESSFUL_0,SUCCESSFUL_1
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [13]:
# Merge one-hot encoded features and drop the originals
charity3_df = charity3_df.merge(encode_df,left_index=True, right_index=True)
charity3_df = charity3_df.drop(charity3_cat,1)
charity3_df.head()

,STATUS,ASK_AMT,APPTYPE_Other,APPTYPE_T10,APPTYPE_T19,APPTYPE_T3,APPTYPE_T4,APPTYPE_T5,APPTYPE_T6,APPTYPE_T7,...,INCAMT_100000-499999,INCAMT_10M-50M,INCAMT_1M-5M,INCAMT_25000-99999,INCAMT_50M+,INCAMT_5M-10M,SPECCONS_N,SPECCONS_Y,SUCCESSFUL_0,SUCCESSFUL_1
0,1,5000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1,108590,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,1,5000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,1,6692,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,1,142590,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [14]:
# Standardize our numerical variables 
# Remove successful target from features data

y = charity3_df.SUCCESSFUL_1
X = charity3_df.drop(columns=["SUCCESSFUL_1","SUCCESSFUL_0"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")


 Random forest predictive accuracy: 0.707


In [19]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  96
hidden_nodes_layer2 = 48
hidden_nodes_layer3 = 24

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
#y_train=y_train.to_numpy()
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/200
25724/25724 [==============================] - 3s 128us/sample - loss: 0.5705 - accuracy: 0.7206
Epoch 2/200
25724/25724 [==============================] - 3s 103us/sample - loss: 0.5557 - accuracy: 0.7305
Epoch 3/200
25724/25724 [==============================] - 3s 107us/sample - loss: 0.5515 - accuracy: 0.7326
Epoch 4/200
25724/25724 [==============================] - 3s 105us/sample - loss: 0.5499 - accuracy: 0.7336
Epoch 5/200
25724/25724 [==============================] - 3s 106us/sample - loss: 0.5482 - accuracy: 0.7341
Epoch 6/200
25724/25724 [==============================] - 3s 102us/sample - loss: 0.5477 - accuracy: 0.7344
Epoch 7/200
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5469 - accuracy: 0.7345
Epoch 8/200
25724/25724 [==============================] - 3s 110us/sample - loss: 0.5464 - accuracy: 0.7348
Epoch 9/200
25724/25724 [==============================] - 3s 106us/sample - loss: 0.5456 - accuracy: 0.7

25724/25724 [==============================] - 3s 102us/sample - loss: 0.5342 - accuracy: 0.7421
Epoch 76/200
25724/25724 [==============================] - 3s 110us/sample - loss: 0.5337 - accuracy: 0.7413
Epoch 77/200
25724/25724 [==============================] - 3s 102us/sample - loss: 0.5341 - accuracy: 0.7417
Epoch 78/200
25724/25724 [==============================] - 3s 100us/sample - loss: 0.5338 - accuracy: 0.7417
Epoch 79/200
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5338 - accuracy: 0.7417
Epoch 80/200
25724/25724 [==============================] - 3s 103us/sample - loss: 0.5341 - accuracy: 0.7417
Epoch 81/200
25724/25724 [==============================] - 3s 99us/sample - loss: 0.5344 - accuracy: 0.7414
Epoch 82/200
25724/25724 [==============================] - 3s 100us/sample - loss: 0.5335 - accuracy: 0.7420
Epoch 83/200
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5334 - accuracy: 0.7419
Epoch 84/200
25724/25724

25724/25724 [==============================] - 3s 101us/sample - loss: 0.5310 - accuracy: 0.7425
Epoch 150/200
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5310 - accuracy: 0.7424
Epoch 151/200
25724/25724 [==============================] - 3s 102us/sample - loss: 0.5307 - accuracy: 0.7427
Epoch 152/200
25724/25724 [==============================] - 3s 105us/sample - loss: 0.5309 - accuracy: 0.7421
Epoch 153/200
25724/25724 [==============================] - 3s 104us/sample - loss: 0.5310 - accuracy: 0.7422
Epoch 154/200
25724/25724 [==============================] - 3s 103us/sample - loss: 0.5313 - accuracy: 0.7427
Epoch 155/200
25724/25724 [==============================] - 3s 104us/sample - loss: 0.5312 - accuracy: 0.7425
Epoch 156/200
25724/25724 [==============================] - 3s 103us/sample - loss: 0.5306 - accuracy: 0.7431
Epoch 157/200
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5310 - accuracy: 0.7428
Epoch 158/200
2

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, <class 'pandas.core.series.Series'>

In [77]:
y = charity3_df.SUCCESSFUL_1.values
X = charity3_df.drop(columns=["SUCCESSFUL_1","SUCCESSFUL_0"]).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [78]:
# Create the SVM model
from sklearn.svm import SVC

svm = SVC(kernel='linear')

In [ ]:
# Train the model
svm.fit(X_train, y_train)

In [ ]:
# Evaluate the model
y_pred = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred):.3f}")